<a href="https://colab.research.google.com/github/nicosesma/otc_strategies/blob/main/RSI_Divergence_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

class RSIDivergenceStrategy:
    """
    Implements a trading strategy based on RSI divergence signals.
    """

    def __init__(self, rsi_period=14):
        """
        Initializes the strategy with a configurable RSI period.
        """
        self.rsi_period = rsi_period

    def _calculate_rsi(self, data):
        """
        A helper method to calculate the RSI for a given price series.
        """
        delta = data.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=self.rsi_period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=self.rsi_period).mean()

        with np.errstate(divide='ignore', invalid='ignore'):
            rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    def generate_signals(self, dataframe: pd.DataFrame):
        """
        Generates buy and sell signals based on RSI divergence.
        """
        dataframe['rsi'] = self._calculate_rsi(dataframe['close'])
        dataframe['buy_signal'] = 0
        dataframe['sell_signal'] = 0

        # We need a decent amount of data to check for divergence
        if len(dataframe) < self.rsi_period * 2:
            return dataframe

        for i in range(self.rsi_period, len(dataframe)):
            # Bullish Divergence (Price makes lower low, RSI makes higher low)
            if (dataframe['close'].iloc[i] < dataframe['close'].iloc[i-1] and
                dataframe['close'].iloc[i-1] < dataframe['close'].iloc[i-2] and
                dataframe['rsi'].iloc[i] > dataframe['rsi'].iloc[i-1]):
                dataframe.loc[dataframe.index[i], 'buy_signal'] = 1

            # Bearish Divergence (Price makes higher high, RSI makes lower high)
            if (dataframe['close'].iloc[i] > dataframe['close'].iloc[i-1] and
                dataframe['close'].iloc[i-1] > dataframe['close'].iloc[i-2] and
                dataframe['rsi'].iloc[i] < dataframe['rsi'].iloc[i-1]):
                dataframe.loc[dataframe.index[i], 'sell_signal'] = 1

        return dataframe

def run_backtest(dataframe):
    """
    Simulates trades and calculates the ROI.
    """
    initial_capital = 10000
    capital = initial_capital
    position_size = 0
    trade_count = 0

    for i in range(len(dataframe)):
        row = dataframe.iloc[i]

        # Entry logic
        if row['buy_signal'] == 1 and position_size == 0:
            position_size = capital / row['close']
            capital = 0
            trade_count += 1
            print(f"Buying at {row['close']:.2f} on {row.name.date()}")

        # Exit logic
        elif row['sell_signal'] == 1 and position_size > 0:
            capital = position_size * row['close']
            position_size = 0
            print(f"Selling at {row['close']:.2f} on {row.name.date()}")

    # Calculate final ROI
    final_capital = capital + (position_size * dataframe['close'].iloc[-1])
    roi = ((final_capital - initial_capital) / initial_capital) * 100

    return roi, trade_count

def generate_mock_data():
    """
    Generates simulated price data with a period of divergence.
    """
    start_date = datetime(2024, 1, 1)
    date_range = pd.date_range(start=start_date, periods=200, freq='D')

    # Create a base price trend
    np.random.seed(42)
    initial_price = 100
    price_changes = np.random.normal(0, 0.5, 200) / 100
    price_series = initial_price * (1 + price_changes).cumprod()

    # Create a bullish divergence
    price_series[50:60] *= np.linspace(1, 0.95, 10) # Price makes a lower low
    price_series[60:70] *= np.linspace(0.95, 0.98, 10) # Price recovers slightly

    df = pd.DataFrame(price_series, index=date_range, columns=['close'])
    return df

if __name__ == '__main__':
    # Generate data and run the backtest
    mock_data = generate_mock_data()
    strategy = RSIDivergenceStrategy()
    signals = strategy.generate_signals(mock_data)

    print("--- Running